# 2.2 CTEs y Recursividad

**Resumen Ejecutivo**: Las Common Table Expressions (CTEs) estructuran consultas complejas en bloques legibles y reutilizables; las CTEs recursivas permiten procesar jerarquías y niveles agregados sin procedimientos iterativos.

---

## 🎯 Objetivos de Aprendizaje
- Refactorizar consultas complejas con CTEs para legibilidad y mantenibilidad.
- Combinar múltiples CTEs (ingresos, margen) y aplicar ranking/filtrado.
- Entender la sintaxis básica de CTEs recursivas para jerarquías.

## 🧰 Prerrequisitos
- Haber completado `01_fundamentos_window_functions.ipynb`.
- Conocer subconsultas y agregaciones avanzadas.
- Familiaridad con estructuras jerárquicas conceptuales (categorías, organigramas).

---

## 📚 Conceptos Clave
- **CTE**: subconsulta nombrada (`WITH nombre AS (...)`) que mejora legibilidad.
- **CTEs encadenadas**: varias CTEs que se referencian entre sí para modularizar lógica.
- **CTE recursiva**: CTE que se llama a sí misma para procesar niveles (ej. árbol de categorías).
- **Caso base + recursión**: patrón estándar para jerarquías (nivel 0, luego unir con hijos).

> ⚠️ CTEs no usados penalizan legibilidad; CTEs recursivas sin límite pueden causar loops infinitos (añadir `MAXRECURSION`).

In [ ]:
-- CTE para margen por producto y filtro posterior
WITH margen AS (
    SELECT producto_id, SUM(cantidad*(p.precio_unitario-p.costo_unitario)) AS margen_total
    FROM dbo.fact_ventas fv
    JOIN dbo.dim_productos p ON fv.producto_id = p.producto_id
    GROUP BY producto_id
)
SELECT * FROM margen WHERE margen_total > 500;

-- Ejemplo conceptual recursivo (estructura jerárquica hipotética de categorías)

In [ ]:
-- Si existiera dim_categorias (id, parent_id, nombre) podríamos: 
-- WITH rec AS (
--   SELECT id, parent_id, nombre, 0 AS nivel FROM dim_categorias WHERE parent_id IS NULL
--   UNION ALL
--   SELECT c.id, c.parent_id, c.nombre, r.nivel+1
--   FROM dim_categorias c
--   JOIN rec r ON c.parent_id = r.id
-- )
-- SELECT * FROM rec;

🟢 Ejercicio: Crear CTE que calcule ingresos por producto y luego filtre top 3 (sin usar TOP, utilizar RANK()).

In [ ]:
WITH ingresos AS (
    SELECT p.producto_id, p.nombre, SUM(fv.cantidad*p.precio_unitario) AS ingresos
    FROM dbo.fact_ventas fv
    JOIN dbo.dim_productos p ON fv.producto_id = p.producto_id
    GROUP BY p.producto_id, p.nombre
), ranked AS (
    SELECT *, RANK() OVER (ORDER BY ingresos DESC) AS rk FROM ingresos
)
SELECT producto_id, nombre, ingresos FROM ranked WHERE rk <= 3;

🟠 Ejercicio: Añadir otra CTE para margen y combinar con ingresos (JOIN interno por producto).

In [ ]:
WITH ingresos AS (
    SELECT p.producto_id, SUM(fv.cantidad*p.precio_unitario) AS ingresos
    FROM dbo.fact_ventas fv
    JOIN dbo.dim_productos p ON fv.producto_id = p.producto_id
    GROUP BY p.producto_id
), margen AS (
    SELECT p.producto_id, SUM(fv.cantidad*(p.precio_unitario-p.costo_unitario)) AS margen
    FROM dbo.fact_ventas fv
    JOIN dbo.dim_productos p ON fv.producto_id = p.producto_id
    GROUP BY p.producto_id
)
SELECT i.producto_id, i.ingresos, m.margen
FROM ingresos i JOIN margen m ON i.producto_id = m.producto_id;

🔴 Reto: Simular jerarquía sin tabla recursiva creando niveles agregados (ej: segmentos -> producto) y sumar unidades por nivel en CTE escalonadas.

Errores comunes: repetir lógica en múltiples CTE sin necesidad; olvidar clausula final; crear CTEs no usados (penaliza legibilidad).

# Créditos

Este material fue revisado y enriquecido parcialmente mediante asistencia de IA (OpenAI y Claude); la validación y decisiones editoriales finales son humanas.

---
## Navegación
[⬅️ Anterior](01_fundamentos_window_functions.ipynb) | [Siguiente ➡️](03_subconsultas_avanzadas.ipynb)
---
